In [53]:
import numpy as np
import fiona
import pprint
from madmex.models import PredictObject
from madmex.models import PredictClassification, Region, ValidClassification
from madmex.util import chunk
import json
from shapely.geometry import mapping
from madmex.util.spatial import geometry_transform
from shapely.geometry import shape
from dask.distributed import Client
import os
from operator import itemgetter
from django.contrib.gis.geos.geometry import GEOSGeometry


In [2]:
name_predict = 'lc_rf_s1_2_20m_resampled_10m_Jalisco_recipe_2017'

In [3]:
qs_ids = PredictClassification.objects.filter(name=name_predict).distinct('predict_object_id')

In [4]:
len(qs_ids)

56

In [8]:
list_ids = [x.predict_object_id for x in qs_ids]

In [5]:
region = 'Jalisco'

In [6]:
region_geom = Region.objects.get(name=region).the_geom
region_geojson = region_geom.geojson
geometry = json.loads(region_geojson)
proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
geometry_region_proj = geometry_transform(geometry,proj4)

In [45]:
list_ids[0]

169

In [50]:
len(list_ids)

56

In [46]:
validation_set='validation_Jalisco'

In [90]:
list_qs = []
proj4_out = '+proj=longlat'
proj4_in = '+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
validation_set='validation_Jalisco'

In [91]:
for id in list_ids:
    seg = PredictObject.objects.filter(id=id)
    path = seg[0].path
    poly = seg[0].the_geom
    poly_geojson = poly.geojson
    geometry = json.loads(poly_geojson)
    geometry_proj = geometry_transform(geometry,proj4_out,crs_in=proj4_in)
    poly_proj = GEOSGeometry(json.dumps(geometry_proj))
    query_set = ValidClassification.objects.filter(valid_object__the_geom__contained=poly_proj,
                                               valid_set=validation_set).prefetch_related('valid_object', 'valid_tag')
    list_qs.append(query_set)

In [92]:
list_qs

[<QuerySet [<ValidClassification: ValidClassification object (268)>, <ValidClassification: ValidClassification object (367)>, <ValidClassification: ValidClassification object (385)>]>,
 <QuerySet []>,
 <QuerySet []>,
 <QuerySet [<ValidClassification: ValidClassification object (301)>, <ValidClassification: ValidClassification object (302)>, <ValidClassification: ValidClassification object (460)>, <ValidClassification: ValidClassification object (461)>, <ValidClassification: ValidClassification object (473)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (468)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (519)>, <ValidClassification: ValidClassification object (520)>]>,
 <QuerySet [<ValidClassification: ValidClassification object (324)>, <ValidClassification: ValidClassification object (327)>, <ValidClassification: ValidClassification object (328)>, <ValidClassification: ValidClassification object (337)>, <ValidClassification: ValidClassification obj